In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [4]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
import os
embeddings_index = {}
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
EMBEDDING_DIM = 50
MAX_SEQUENCE_LENGTH = 200
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

W0727 17:39:08.725967 140134923257664 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [ ]:
pad_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
pad_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
model.add(LSTM(128, dropout=.25, recurrent_dropout=.25))
model.add(Dropout(.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

W0727 17:39:10.445752 140134923257664 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 17:39:10.449728 140134923257664 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 17:39:10.483816 140134923257664 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0727 17:39:10.491359 140134923257664 deprecation_wrapper.py:119] From /home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please us

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
check = ModelCheckpoint('rnn50.hdf5', verbose=1, save_best_only=True)
es = EarlyStopping(verbose=1, patience=3)

In [ ]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=15, batch_size=128, validation_data=(pad_test, y_test), callbacks=[check, es])

Train on 50000 samples, validate on 2000 samples
Epoch 1/15
35712/50000 [====================>.........] - ETA: 8:12 - loss: 0.5677 - acc: 0.7045

In [ ]:
accuracy_score(y_test, model.predict_classes(pad_test))